# Flow Routing Lesson 1.
You can run this notebook on your laptop with landlab v2.6.0 or in lab.openearthscape using the CSDMS environment.

# 1. Steepest descent flow direction and downstream accumulation on a simple NetworkModelGrid.
- Let's start by a grid containing two simple, separate linked network of nodes. Links could be rivers and nodes gauge stations.

In [ ]:
import numpy as np # import package to handle arrays
from landlab import NetworkModelGrid # import the network grid class.
from landlab.plot import graph # import class for detailed plot of the grid.

# Set the parameters of the grid.
grid_param = {
    "yx_of_node": (
        (0, 100, 200, 200, 300, 400, 400, 125, 300, 280),
        (0, 0, 100, -50, -100, 50, -150, -100, 130, 80),
    ), # position of nodes.
    "links": ((1, 0), (2, 1), (1, 7), (3, 1), (3, 4), (4, 5), (4, 6),
              (8, 9), 
             ), # link between nodes.
    "xy_axis_units": "m",
}

# Creation of the grid
g = NetworkModelGrid(**grid_param)

# Display grid.
Fig = graph.plot_graph(g, at="node,link")
import matplotlib
import matplotlib.pyplot as plt
plt.savefig('fig/01_plain_network_grid.png')

# Display prerecorded image.
from IPython import display
display.Image("fig/01_01_plain_network_grid.png")

- We set the base-level nodes #0 and #7, close the node #8 (because we want to!) and let's see what are the active links with their head and tail nodes.

In [ ]:
# Set of perimeter base-level nodes
g.status_at_node[[0, 7]] = g.BC_NODE_IS_FIXED_VALUE
# Close one node.
g.status_at_node[8] = g.BC_NODE_IS_CLOSED

print("Status at node: ", g.status_at_node)

# Active links and head and tail nodes.
active_links = g.active_links
src_nodes = g.node_at_link_head[active_links]
dst_nodes = g.node_at_link_tail[active_links]
print("Active links: ", active_links)
print("Head nodes of links:", src_nodes)
print("Tail nodes of links:", dst_nodes)

# Display prerecorded image.
from IPython import display
display.Image("fig/01_02_plain_network_grid.png")

- We add a topography, such as each link has consistent downstream direction with the other links (in other words we avoid depressions).

In [ ]:
# Create field topographic elevation at nodes without depression.
z = g.add_field("topographic__elevation", 
                [1., 5., 6., 7., 16., 10., 10., 2., 14., 16. ],
                at="node", clobber="True") # clobber allows to erase previous field of same name if existed.
print("Nodes: ", np.arange(g.number_of_nodes))
print("Elevation (m):", z)

# Display prerecorded image.
display.Image("fig/01_03_plain_network_grid_topography.png")

- We will now determine by "hand" the receiver of each node, according the Steepest descent algorithm (O'Callaghan and Mark, 1984). Of course we already know the result since we designed the grid to be trivial.
    - Let's start to calculate the topographic gradients (=slopes) between the nodes.

In [ ]:
# Gradients of topographic__elevation.
link_slope = g.calc_grad_at_link(g.at_node["topographic__elevation"])
link_slope = link_slope[g.active_links]
print("Links: ", g.active_links)
print("Gradients (slopes) at link: ", link_slope)

-
    - Then we initialize some fields for steepest descent calculation, we set the boundary nodes as their own receivers and run the algorithm. Results are the receiver corresponding to each node.

In [ ]:
# Steepest algorithm.

# Initialize fields.
receiver_link = -1 * np.ones(g.number_of_nodes, dtype='int');
receiver = -1 * np.ones(g.number_of_nodes, dtype='int');
steepest_slope = g.add_zeros("topographic__steepest_slope", at="node", clobber="True")
receiver_link = g.add_field("flow__link_to_receiver_node", receiver_link, at="node", clobber="True")
receiver =  g.add_field("flow__receiver_node", receiver, at="node", clobber="True")

# Set receivers of nodes in boundary or closed to the nodes themselves.
receiver[g.status_at_node != 0] = np.where(g.status_at_node != 0)[0];

# Run the steepest descent algorithm.
for i in range(len(active_links)):
    src_id = src_nodes[i]
    dst_id = dst_nodes[i]
    if z[src_id] > z[dst_id] and link_slope[i] > steepest_slope[src_id]:
        receiver[src_id] = dst_id
        steepest_slope[src_id] = link_slope[i]
        receiver_link[src_id] = active_links[i]
    elif z[dst_id] > z[src_id] and -link_slope[i] > steepest_slope[dst_id]:
        receiver[dst_id] = src_id
        steepest_slope[dst_id] = - link_slope[i]
        receiver_link[dst_id] = active_links[i]
        
# Print steepest descent results.
print("Donor nodes: ", np.arange(g.number_of_nodes))
print("Receiver nodes: ", g.at_node["flow__receiver_node"])

- This can be performed in a more straightforward way by using the Steepest descent component of Landlab, **FlowDirectorSteepest**.

In [ ]:
# Use of landlab FlowDirectorSteepest component.

# Reset fields.
link_slope = []
receiver_link = []
receiver = []
src_nodes = []
dst_nodes = []
steepest_slope = []

# Run component
from landlab.components import FlowDirectorSteepest       # Import component.
fd = FlowDirectorSteepest(g, "topographic__elevation")    # Instantiate component on the specific topographic layer of the grid.
fd.run_one_step();                                         # Run the steepest descent algorithm. We can also use fd.direct_flow();

# Result display.
print("Donor nodes: ", np.arange(g.number_of_nodes))
print("Receiver nodes: ", g.at_node["flow__receiver_node"])

- So, you may say: wow, the landlab FlowDirectorSteepest component just does the same thing as what i just coded in this notebook! The answer is yes, but in an optimized way:
    - The problem with the code we wrote is that it's a loop. For large grids, loops are extremely time-consuming in python. The FlowDirectorSteepest component uses a version of this loop (1) translated in C-language using Cython and (2) compiled when you install Landlab. Loops are performant in C because data access in arrays do not use objects as in python. So the FlowDirectorSteepest component works similarly to methods of the Numpy package, which similarly encapsulates C-code within python wrappers to optimize classic handling of arrays operations.
<br /> 
<br /> 
- Now that we have the donor-receiver relationships, we can construct the way the discharge goes downstream. Again, we know the result since we design the grid trivial on purpose. Let's do this using the FlowAccumulator component which implements the Braun and Willet, 2013's upstream/downstream O(n) algorithm. 
    - This algorithm starts from the most elevated nodes and constructs the network downstream, using a stack and a recursive function.
<br />  
<br />  
- To do this:
    - we have to add first the field cell_area_at_node to the NetworkModelGrid (this is not necessary for classic grids, but the poor sibling has limited functionalities in Landlab right now (2023)).
    - we also add a uniform rainfall input of 2 to the grid, through the water__unit_flux_in field.

In [ ]:
# Use of the FlowAccumulator component, which implements Braun and Willet, 2013's upstream/downstream O(n) algorithm.

# Add the fields for drainage and discharge.
g.add_ones("cell_area_at_node", at="node", clobber="True")
g.add_field("water__unit_flux_in", 2. * np.ones(g.number_of_nodes), at="node", clobber="True")

# Then import FlowAccumulater, instantiate and run.
from landlab.components import FlowAccumulator
fa = FlowAccumulator(g, 'topographic__elevation', flow_director=fd);
fa.run_one_step();

# Result display.
print("Drainage area: ", g.at_node['drainage_area'])
print("River discharge: ", g.at_node['surface_water__discharge'])

# Display prerecorded image.
display.Image("fig/01_04_plain_network_grid_discharge.png")

- We can also code and run this more simply, without specifically instantiating and running a FlowDirectorSteepest before instantiating and running the FlowAccumulator:

```
fa = FlowAccumulator(g, 'topographic__elevation', flow_director=FlowDirectorSteepest)
fa.run_one_step()
```


# Flow direction and downstream accumulation on a less trivial grid.
- Let's imagine that the network is more complicated. Suppose that the node #4 covers 5 km^2. Given its high elevation, water from node #4 could flow to nodes #0, #3, and #6.

In [ ]:
# flow from node 4 is split in 4.
grid_param["links"] = ((1, 0), (2, 1), (1, 7), (3, 1), (3, 4), (4, 5), (4, 6),
              (8, 9),               
              (0,2), (2, 4), (3, 2)) # link between nodes.

# Creation of the grid
g = NetworkModelGrid(**grid_param)

# Set of perimeter base-level nodes
g.status_at_node[[0, 7]] = g.BC_NODE_IS_FIXED_VALUE

# Display grid.
graph.plot_graph(g, at="node,link")

# Create field topographic elevation at nodes without depression.
z = g.add_field("topographic__elevation", 
                [1., 5., 6., 7., 16., 10., 10., 2., 14., 16. ],
                at="node", clobber="True")

# Add the fields for drainage and discharge.
g.add_ones("cell_area_at_node", at="node", clobber="True")
g.add_field("water__unit_flux_in", 2. * np.ones(g.number_of_nodes), at="node", clobber="True")

# Flow direction and accumulation with Steepest descent.
fa = FlowAccumulator(g, "topographic__elevation", flow_director=FlowDirectorSteepest)
fa.run_one_step();

# Result display.
print("Donor nodes: ", np.arange(g.number_of_nodes))
print("Receiver nodes: ", g.at_node["flow__receiver_node"])
print("Drainage area: ", g.at_node['drainage_area'])
print("River discharge: ", g.at_node['surface_water__discharge'])

# Display grid.
Fig = graph.plot_graph(g, at="node,link")
import matplotlib
import matplotlib.pyplot as plt
plt.savefig('fig/02_plain_network_grid.png')

# Display prerecorded image.
display.Image("fig/02_01_plain_network_grid.png")

- Despite connections between #4 and several other nodes, the FlowDirectorSteepest still favors the connection between #4 downstream to #1 because it is the steepest slope. This leads to two remarks:
    - the FlowDirectorSteepest applies the steepest slope for all nodes of the network. Some other flow routing components, such as the PriorityFloodFlowRouter component, based on Barnes et al., 2014, although still based on steepest descent, can give a result that doesn't strictly apply the steepest descent for all nodes.
    - you cannot rely on the Steepest Descent algorithm to give a drainage network corresponding to what is actually observed. This algorithm is only an algorithm to get a drainage network at first order within a reduced amount of execution time.
    - theFlowDirectorSteepest directs donors to receivers in a one-to-one way. Other algorithms exist to implement this in a one-to-multiple nodes way, for instance:
        - the Multiple Flow Direction (MFD) algorithm of [Quinn et al., 1991](https://doi.org/10.1002/hyp.3360050106), implemented in the **FlowDirectorMFD** component. This component works for all grids except the NetworkModelGrid.
        - the one-to-two node D-Infinity (DINF) algorithm of [Tarboton, 1997](https://doi.org/10.1029/96WR03137), implemented in the **FlowDirectorDinf** component. That component only works for RasterModelGrids.
<br />
<br />
- Let's imagine now that the data indicate a depression in nodes #3 and #6.

In [ ]:
# flow from node 4 is split in 4.
grid_param["links"] = ((1, 0), (2, 1), (1, 7), (3, 1), (3, 4), (4, 5), (4, 6),
              (8, 9)) # link between nodes.

# Creation of the grid
g = NetworkModelGrid(**grid_param)

# Set of perimeter base-level nodes
g.status_at_node[[0, 7]] = g.BC_NODE_IS_FIXED_VALUE

# Create field topographic elevation at nodes without depression.
z = g.add_field("topographic__elevation", 
                [1., 5., 6., 4., 16., 10., 4.5, 2., 14., 16. ],
                at="node", clobber="True")

# Add the fields for drainage and discharge.
g.add_ones("cell_area_at_node", at="node", clobber="True")
g.add_field("water__unit_flux_in", 2. * np.ones(g.number_of_nodes), at="node", clobber="True")

# Flow direction and accumulation with Steepest descent.
fa = FlowAccumulator(g, "topographic__elevation", flow_director=FlowDirectorSteepest)
fa.run_one_step();

# Result display.
print("Donor nodes: ", np.arange(g.number_of_nodes))
print("Receiver nodes: ", g.at_node["flow__receiver_node"])
print("Drainage area: ", g.at_node['drainage_area'])
print("River discharge: ", g.at_node['surface_water__discharge'])

# Display prerecorded image.
display.Image("fig/02_02_plain_network_grid_depression.png")

- The FlowDirectorSteepest and FlowAccumulator components cannot solve this depression and redirect the flow by themselves. 
    - This have even a bigger impact of landscape evolution models, in which depressions are continuously created by processes such as landslides or ice flow. We sometimes need to breach or fill the depressions because they may not correspond to something realistic or create some artefacts in the landscape evolution models.
<br />
<br />
- We could apply the DepressionFinderAndRouter component on the grid after having run the FlowDirectorSteepest and FlowAccumulator component so that the flow manages to breach the depression outlet. Unfortunately, the DepressionFinderAndRouter still doesn't handle NetworkModelGrid.

<br/>
Please reuse this material citing Lenard et al., 2023, CSDMS Annual Meeting.<br/>
Author: Sebastien Lenard sebastien.lenard@gmail.com<br/>
Date: 2023, May.

### Click here to learn about <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>